In [ ]:
#importando as bibliotecas
import pandas as pd  #bibioteca responsável para o tratamento e limpeza dos dados
import numpy as np #biblioteca utilizada para o tratamento eficiente de dados numéricos
import datetime  #biblioteca utilizada para trabalhar com datas
from matplotlib import pyplot as plt  #plotar os gráficos
import seaborn as sns #plot de gráficos

In [ ]:
#carregando os dados para o pandas
df_consultas=pd.read_csv('data.csv')

In [ ]:
df_consultas.head(10)

In [ ]:
df_consultas.shape

In [ ]:
df_consultas.info()

In [ ]:
df_consultas.isnull().sum()

In [ ]:
df_consultas['No-show'].value_counts() # Yes o paciente não compareceu / No o paciente compareceu

In [ ]:
df_consultas['No-show'].value_counts()['No']/len(df_consultas['No-show'])

In [ ]:
#analisando as "estatísticas do dataset"
df_consultas.describe()

In [ ]:
#contando a quantidade de valores distintos em cada uma das colunas
for colunas in list(df_consultas.columns):
    print( "{0:25} {1}".format(colunas, df_consultas[colunas].nunique()) )



Em quantas localidades diferentes (Neighbourhood) os pacientes residem?

Comparando a identificação do paciente (PatientId) com o número dos agendamentos das consultas (AppointmentID) o que podemos inferir?

Quantas variáveis binárias (apenas dois valores) existem no dataset?


In [ ]:
#contando quantas idades diferentes existem no df
df_consultas['Age'].nunique()

In [ ]:
#contando quantas idades diferentes existem no df
df_consultas['Age'].min()

In [ ]:
df_consultas['Age'].max()

In [ ]:
#plotando o histograma de algumas variáveis 
df_consultas['Age'].hist(bins=len(df_consultas['Age'].unique()))

In [ ]:
df_consultas['SMS_received'].hist(bins=len(df_consultas['SMS_received'].unique()))

In [ ]:
df_consultas['SMS_received'].unique()

In [ ]:
#criando uma nova coluna -> Tempo de espera (diferença entre a data em que a consulta foi agendada e o dia da consulta)
df_consultas.ScheduledDay=pd.to_datetime(df_consultas.ScheduledDay)  #transformando as colunas par o tipo datetime
df_consultas.AppointmentDay=pd.to_datetime(df_consultas.AppointmentDay)

In [ ]:
#Encontra a diferença entre o momento da marcação da consulta e o dia da consulta
tempo_espera=df_consultas.ScheduledDay-df_consultas.AppointmentDay



In [ ]:
tempo_espera[:10]

In [ ]:
df_consultas['AwaitingTime']=tempo_espera.apply(lambda x: x.days) #transforma os valores em dias

In [ ]:
df_consultas.head(8)

In [ ]:
df_consultas.info()

In [ ]:
#Encontrando as idades negativas
df_consultas[df_consultas['Age'] < 0]['Age'].value_counts()

In [ ]:
#transformando os tempo de espera para um valor não negativo
df_consultas_filtrado['AwaitingTime'] = df_consultas_filtrado['AwaitingTime'].apply(lambda x: abs(x))

In [ ]:
df_consultas_filtrado.head()

In [ ]:
#aplicando a transformação para os dados categóricos
categoricas=['Neighbourhood','Gender','No-show']
for coluna in categoricas:
  df_consultas_filtrado[coluna]=pd.Categorical(df_consultas_filtrado[coluna]).codes



In [ ]:
df_consultas_filtrado.head(8)

In [ ]:
#Qual o tipo de transformação foi utilizada?
#analisando os SMS enviados e o número de vezes que o paciente compareceu ou não 
sms_x_comparecer = df_consultas_filtrado.groupby(['SMS_received', 'No-show'])['SMS_received'].count().unstack('No-show').fillna(0)
sms_x_comparecer[[0, 1]].plot(kind='bar', stacked=True) 
plt.title('Analise do número de SMS recebido e se a paciente compareceu ou não à consulta') 
plt.xlabel('Numero de SMS recebidos') 
plt.ylabel('Frequência')

In [ ]:
sms_x_comparecer

In [ ]:
9784/(25698+9784)

In [ ]:
#Qual é a proporção de pacientes que receberam o sms e NÃO compareceram?


#plotando o número de consultas por região 
regioes = df_consultas_filtrado['Neighbourhood'].unique()
plt.figure(figsize=(22,10))
ax = sns.countplot(x='Neighbourhood', data=df_consultas_filtrado, order=regioes)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right", fontsize=8)
plt.title('Consultas por região', fontsize=14, fontweight='bold')
plt.show()



In [ ]:
#Qual região possui o maior número de consultas marcadas?
#selecionando os dados para a construção da previsão
entradas = ['Gender','Age','Neighbourhood','Scholarship','Hipertension','Diabetes','Alcoholism','SMS_received','AwaitingTime']
saida=['No-show']

In [ ]:
x=df_consultas_filtrado[entradas]
y=df_consultas_filtrado[saida]



In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)



In [ ]:
x_train.shape

In [ ]:
#aplicando um modelo de classificação via árvore de decisão
from sklearn.tree import DecisionTreeClassifier
clf_arvore = DecisionTreeClassifier() 
clf_arvore.fit(x_train, y_train)

In [ ]:
#realiza a previsão com os dados
y_previsto = clf_arvore.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
acuracia = accuracy_score(y_test, y_previsto)
print('Acurácia da àrvore de Decisão: ',acuracia)

In [ ]:
#contrução da matriz de confusão
from sklearn.metrics import classification_report, confusion_matrix
matriz_confusao = confusion_matrix(y_test, y_previsto)
print(matriz_confusao)

In [ ]:
#realiza o plot da matriz de confusão
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=matriz_confusao)
plt.show()